In [ ]:
!wget https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv -O features.csv
!wget https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv -O labels.csv
!pip install pyspark 

--2021-05-26 17:43:14--  https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.74.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.74.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20069199 (19M) [text/csv]
Saving to: ‘features.csv’

features.csv        100%[===================>]  19.14M  20.5MB/s    in 0.9s    

2021-05-26 17:43:16 (20.5 MB/s) - ‘features.csv’ saved [20069199/20069199]

--2021-05-26 17:43:16--  https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.74.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.74.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1148327 (1.1M) [text/csv]
Saving to: ‘labels.csv’

labels.csv          100%[===================>]   1.09M  2.87MB/s    in 0.4s    

2021-05-26 17:43:16 (2.87 MB

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pandas as pd 
from pyspark.sql.functions import year, month

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql.functions import col

In [ ]:
!ls

drive  features.csv  labels.csv  sample_data


In [ ]:
sc = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
features = sc.read.csv('features.csv', inferSchema=True, 
                       header=True)
label = sc.read.csv('labels.csv', inferSchema=True, 
                       header=True)

In [ ]:
type(features)

pyspark.sql.dataframe.DataFrame

In [ ]:
print(features.count())
print(label.count())
print(features.columns)
print(label.columns)

59400
59400
['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'population', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'construction_year', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']
['id', 'status_group']


In [ ]:
features.toPandas().head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,None,GeoData Consultants Ltd,Other,None,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,None,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,None,None,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [ ]:
data = features.join(label, on =('id'))

In [ ]:
data.show(5)

+-----+----------+-------------+------------+----------+------------+-----------+------------+--------------------+-----------+--------------------+----------+-------+-----------+-------------+---------+----------+----------+--------------+--------------------+-----------------+--------------------+------+-----------------+---------------+---------------------+---------------------+----------+----------------+--------------+------------+-------------+-------------+------------+--------------+--------------------+--------------------+------------+--------------------+---------------------+--------------+
|   id|amount_tsh|date_recorded|      funder|gps_height|   installer|  longitude|    latitude|            wpt_name|num_private|               basin|subvillage| region|region_code|district_code|      lga|      ward|population|public_meeting|         recorded_by|scheme_management|         scheme_name|permit|construction_year|extraction_type|extraction_type_group|extraction_type_class|mana

In [ ]:
print(data.printSchema())

root
 |-- id: integer (nullable = true)
 |-- amount_tsh: double (nullable = true)
 |-- date_recorded: string (nullable = true)
 |-- funder: string (nullable = true)
 |-- gps_height: integer (nullable = true)
 |-- installer: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- wpt_name: string (nullable = true)
 |-- num_private: integer (nullable = true)
 |-- basin: string (nullable = true)
 |-- subvillage: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: integer (nullable = true)
 |-- district_code: integer (nullable = true)
 |-- lga: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- public_meeting: boolean (nullable = true)
 |-- recorded_by: string (nullable = true)
 |-- scheme_management: string (nullable = true)
 |-- scheme_name: string (nullable = true)
 |-- permit: boolean (nullable = true)
 |-- construction_year: integer (nullable = 

In [ ]:
data.show(10)

+-----+----------+-------------+-------------------+----------+------------+-----------+------------+--------------------+-----------+--------------------+---------------+---------+-----------+-------------+---------------+----------+----------+--------------+--------------------+-----------------+--------------------+------+-----------------+---------------+---------------------+---------------------+----------+----------------+--------------+------------+-------------+-------------+------------+--------------+--------------------+--------------------+------------+--------------------+---------------------+--------------+
|   id|amount_tsh|date_recorded|             funder|gps_height|   installer|  longitude|    latitude|            wpt_name|num_private|               basin|     subvillage|   region|region_code|district_code|            lga|      ward|population|public_meeting|         recorded_by|scheme_management|         scheme_name|permit|construction_year|extraction_type|extracti

In [ ]:
#изменение типа колонки на строку
data = data.withColumn('region_code', col('region_code').cast(StringType()))\
.withColumn('district_code', col('district_code').cast(StringType()))

In [ ]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- amount_tsh: double (nullable = true)
 |-- date_recorded: string (nullable = true)
 |-- funder: string (nullable = true)
 |-- gps_height: integer (nullable = true)
 |-- installer: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- wpt_name: string (nullable = true)
 |-- num_private: integer (nullable = true)
 |-- basin: string (nullable = true)
 |-- subvillage: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- district_code: string (nullable = true)
 |-- lga: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- public_meeting: boolean (nullable = true)
 |-- recorded_by: string (nullable = true)
 |-- scheme_management: string (nullable = true)
 |-- scheme_name: string (nullable = true)
 |-- permit: boolean (nullable = true)
 |-- construction_year: integer (nullable = tr

In [ ]:
#убрать дубли по полю id
data = data.dropDuplicates(['id'])

In [ ]:
#избавиться от линних пробелов 
str_cols = [item[0] for item in data.dtypes if item [1].startswith('string')]
for cols in str_cols:
  data = data.withColumn(cols, trim(data[cols]))

In [ ]:
#проверки на пропуски в столбцах
data.select(count(when(isnan('funder'), 'funder'))).alias('funder').show()

+----------------------------------------------+
|count(CASE WHEN isnan(funder) THEN funder END)|
+----------------------------------------------+
|                                             0|
+----------------------------------------------+



In [ ]:
data.select(count(when(col('funder').isNull(), 'funder'))).show()

+-------------------------------------------------+
|count(CASE WHEN (funder IS NULL) THEN funder END)|
+-------------------------------------------------+
|                                             3635|
+-------------------------------------------------+



In [ ]:
for c in data.columns:
  if c not in ['date_recorded', 'public_meeting', 'permit']:
    data.select([count(when(isnan(c)|col(c).isNull(), c)).alias(c)]).show()

+---+
| id|
+---+
|  0|
+---+

+----------+
|amount_tsh|
+----------+
|         0|
+----------+

+------+
|funder|
+------+
|  3635|
+------+

+----------+
|gps_height|
+----------+
|         0|
+----------+

+---------+
|installer|
+---------+
|     3655|
+---------+

+---------+
|longitude|
+---------+
|        0|
+---------+

+--------+
|latitude|
+--------+
|       0|
+--------+

+--------+
|wpt_name|
+--------+
|       0|
+--------+

+-----------+
|num_private|
+-----------+
|          0|
+-----------+

+-----+
|basin|
+-----+
|    0|
+-----+

+----------+
|subvillage|
+----------+
|       371|
+----------+

+------+
|region|
+------+
|     0|
+------+

+-----------+
|region_code|
+-----------+
|          0|
+-----------+

+-------------+
|district_code|
+-------------+
|            0|
+-------------+

+---+
|lga|
+---+
|  0|
+---+

+----+
|ward|
+----+
|   0|
+----+

+----------+
|population|
+----------+
|         0|
+----------+

+-----------+
|recorded_by|
+-----------+
|     

In [ ]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns if c not in {'date_recorded', 'public_meeting', 'permit'}]).show()

+---+----------+------+----------+---------+---------+--------+--------+-----------+-----+----------+------+-----------+-------------+---+----+----------+-----------+-----------------+-----------+-----------------+---------------+---------------------+---------------------+----------+----------------+-------+------------+-------------+-------------+--------+--------------+------+-----------+------------+---------------+---------------------+------------+
| id|amount_tsh|funder|gps_height|installer|longitude|latitude|wpt_name|num_private|basin|subvillage|region|region_code|district_code|lga|ward|population|recorded_by|scheme_management|scheme_name|construction_year|extraction_type|extraction_type_group|extraction_type_class|management|management_group|payment|payment_type|water_quality|quality_group|quantity|quantity_group|source|source_type|source_class|waterpoint_type|waterpoint_type_group|status_group|
+---+----------+------+----------+---------+---------+--------+--------+----------

In [ ]:
#пропуски в процентном соотношении
data.select([(count(when(isnan(c) | col(c).isNull(), c))/data.count()).alias(c) for c in data.columns if c not in {'date_recorded', 'public_meeting', 'permit'}]).show()

+---+----------+------------------+----------+--------------------+---------+--------+--------+-----------+-----+--------------------+------+-----------+-------------+---+----+----------+-----------+-------------------+------------------+-----------------+---------------+---------------------+---------------------+----------+----------------+-------+------------+-------------+-------------+--------+--------------+------+-----------+------------+---------------+---------------------+------------+
| id|amount_tsh|            funder|gps_height|           installer|longitude|latitude|wpt_name|num_private|basin|          subvillage|region|region_code|district_code|lga|ward|population|recorded_by|  scheme_management|       scheme_name|construction_year|extraction_type|extraction_type_group|extraction_type_class|management|management_group|payment|payment_type|water_quality|quality_group|quantity|quantity_group|source|source_type|source_class|waterpoint_type|waterpoint_type_group|status_group

In [ ]:
agg_row = data.select([(count(when(isnan(c) | col(c).isNull(), c))/data.count()).alias(c) for c in data.columns if c not in {'date_recorded', 'public_meeting', 'permit'}]).collect()

In [ ]:
agg_row[0]

Row(id=0.0, amount_tsh=0.0, funder=0.0611952861952862, gps_height=0.0, installer=0.061531986531986535, longitude=0.0, latitude=0.0, wpt_name=0.0, num_private=0.0, basin=0.0, subvillage=0.0062457912457912455, region=0.0, region_code=0.0, district_code=0.0, lga=0.0, ward=0.0, population=0.0, recorded_by=0.0, scheme_management=0.06526936026936027, scheme_name=0.4741750841750842, construction_year=0.0, extraction_type=0.0, extraction_type_group=0.0, extraction_type_class=0.0, management=0.0, management_group=0.0, payment=0.0, payment_type=0.0, water_quality=0.0, quality_group=0.0, quantity=0.0, quantity_group=0.0, source=0.0, source_type=0.0, source_class=0.0, waterpoint_type=0.0, waterpoint_type_group=0.0, status_group=0.0)

In [ ]:
agg_dict_list = [row.asDict() for row in agg_row]
agg_dict = agg_dict_list[0]

In [ ]:
agg_dict

{'amount_tsh': 0.0,
 'basin': 0.0,
 'construction_year': 0.0,
 'district_code': 0.0,
 'extraction_type': 0.0,
 'extraction_type_class': 0.0,
 'extraction_type_group': 0.0,
 'funder': 0.0611952861952862,
 'gps_height': 0.0,
 'id': 0.0,
 'installer': 0.061531986531986535,
 'latitude': 0.0,
 'lga': 0.0,
 'longitude': 0.0,
 'management': 0.0,
 'management_group': 0.0,
 'num_private': 0.0,
 'payment': 0.0,
 'payment_type': 0.0,
 'population': 0.0,
 'quality_group': 0.0,
 'quantity': 0.0,
 'quantity_group': 0.0,
 'recorded_by': 0.0,
 'region': 0.0,
 'region_code': 0.0,
 'scheme_management': 0.06526936026936027,
 'scheme_name': 0.4741750841750842,
 'source': 0.0,
 'source_class': 0.0,
 'source_type': 0.0,
 'status_group': 0.0,
 'subvillage': 0.0062457912457912455,
 'ward': 0.0,
 'water_quality': 0.0,
 'waterpoint_type': 0.0,
 'waterpoint_type_group': 0.0,
 'wpt_name': 0.0}

In [ ]:
col_null = list({i for i in agg_dict if agg_dict[i] > 0.4})
print(agg_dict)
print(col_null)

{'id': 0.0, 'amount_tsh': 0.0, 'funder': 0.0611952861952862, 'gps_height': 0.0, 'installer': 0.061531986531986535, 'longitude': 0.0, 'latitude': 0.0, 'wpt_name': 0.0, 'num_private': 0.0, 'basin': 0.0, 'subvillage': 0.0062457912457912455, 'region': 0.0, 'region_code': 0.0, 'district_code': 0.0, 'lga': 0.0, 'ward': 0.0, 'population': 0.0, 'recorded_by': 0.0, 'scheme_management': 0.06526936026936027, 'scheme_name': 0.4741750841750842, 'construction_year': 0.0, 'extraction_type': 0.0, 'extraction_type_group': 0.0, 'extraction_type_class': 0.0, 'management': 0.0, 'management_group': 0.0, 'payment': 0.0, 'payment_type': 0.0, 'water_quality': 0.0, 'quality_group': 0.0, 'quantity': 0.0, 'quantity_group': 0.0, 'source': 0.0, 'source_type': 0.0, 'source_class': 0.0, 'waterpoint_type': 0.0, 'waterpoint_type_group': 0.0, 'status_group': 0.0}
['scheme_name']


In [ ]:
#избавляемся от олонок со слишком большим количеством пропусков 
data = data.drop(*col_null)

In [ ]:
data.groupBy('recorded_by').count().show()

+--------------------+-----+
|         recorded_by|count|
+--------------------+-----+
|GeoData Consultan...|59400|
+--------------------+-----+



In [ ]:
data.groupBy('water_quality').count()\
.orderBy('count', ascending = False)\
.show()

+------------------+-----+
|     water_quality|count|
+------------------+-----+
|              soft|50818|
|             salty| 4856|
|           unknown| 1876|
|             milky|  804|
|          coloured|  490|
|   salty abandoned|  339|
|          fluoride|  200|
|fluoride abandoned|   17|
+------------------+-----+



In [ ]:
data = data.drop('recorded_by')

In [ ]:
#сводник
data\
.groupBy('status_group').pivot('region').sum('amount_tsh').show()

+--------------------+---------+-------------+------+---------+------+--------+------------------+-------+--------+--------+-----+--------+-------+-------+---------+--------+---------+---------+--------+------+--------+
|        status_group|   Arusha|Dar es Salaam|Dodoma|   Iringa|Kagera|  Kigoma|       Kilimanjaro|  Lindi| Manyara|    Mara|Mbeya|Morogoro| Mtwara| Mwanza|    Pwani|   Rukwa|   Ruvuma|Shinyanga| Singida|Tabora|   Tanga|
+--------------------+---------+-------------+------+---------+------+--------+------------------+-------+--------+--------+-----+--------+-------+-------+---------+--------+---------+---------+--------+------+--------+
|functional needs ...|  10280.0|          0.0|   0.0| 118400.0|   0.0|270770.0|          129248.0| 3625.0| 71315.0|   900.0|  0.0| 70005.0|36240.0|10000.0|    520.0|292980.0| 125650.0|   2200.0|  6505.0|   0.0|  4310.0|
|          functional|1125485.0|     181175.0|   0.0|5515710.0|   0.0|530725.0|         746942.25|24560.0|312745.0| 9678

In [ ]:
print(str_cols)

['date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward', 'recorded_by', 'scheme_management', 'scheme_name', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'status_group']


In [ ]:
for column in str_cols[:2]:
  print(column)
  print(data.groupBy(column).count().orderBy('count', ascending = False).show())
  values_cat = data.groupBy(column).count().collect()
  lessthan = [x[0] for x in values_cat if x[1] < 1000]
  data = data.withColumn(column, when(data[column].isin(lessthan) ,"others").otherwise(data[column]))
  #data = data.withColumn(column, when(col(column).isin(lessthan), 'Others').overwise(col(column)))
  data.groupBy(column).count().orderBy('count', ascending = False).show()

date_recorded
+-------------+-----+
|date_recorded|count|
+-------------+-----+
|   2011-03-15|  572|
|   2011-03-17|  558|
|   2013-02-03|  546|
|   2011-03-14|  520|
|   2011-03-16|  513|
|   2011-03-18|  497|
|   2011-03-19|  466|
|   2013-02-04|  464|
|   2013-01-29|  459|
|   2011-03-04|  458|
|   2013-02-14|  444|
|   2013-01-24|  435|
|   2011-03-05|  434|
|   2013-02-15|  429|
|   2013-03-15|  428|
|   2011-03-11|  426|
|   2013-01-30|  421|
|   2013-02-16|  418|
|   2011-03-23|  417|
|   2011-03-09|  416|
+-------------+-----+
only showing top 20 rows

None
+-------------+-----+
|date_recorded|count|
+-------------+-----+
|       others|59400|
+-------------+-----+

funder
+--------------------+-----+
|              funder|count|
+--------------------+-----+
|Government Of Tan...| 9084|
|                null| 3635|
|              Danida| 3114|
|              Hesawa| 2202|
|               Rwssp| 1374|
|          World Bank| 1349|
|                Kkkt| 1287|
|        World Visi

In [ ]:
data.groupBy('population').count().orderBy('population').show()

In [ ]:
data.withColumn('population', when(col('population')<2, lit(None))\
  .otherwise(col('population')))\
.groupBy('population').count()\
.orderBy('population').show()


In [ ]:
data = data.withColumn('population', when(col('population')<2, lit(None))\
  .otherwise(col('population')))

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, count, when, isnan, lit, avg
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [ ]:
data.select(avg(data['population'])).show()

+-----------------+
|  avg(population)|
+-----------------+
|344.5708201587404|
+-----------------+



In [ ]:
column = 'population'
w = Window.partitionBy('district_code')
data = data.withColumn(column, when(data[column].isNull() ,avg(data["population"]).over(w)).otherwise(data[column]))